In [63]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit, cross_validate
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import normalize

## Dataset Configuration

In [64]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
#d_genetic = pd.read_csv(os.path.join('data', 'dataset_genetic_cleaned_noOHE.csv'))
#d_vampire = pd.read_csv(os.path.join('data', 'dataset_vampire_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

In [65]:
C = d_clinical.values
#G = d_genetic.values
#V = d_vampire.values

In [66]:
y_d = outputs["dement_fail"].values
y_c = outputs["cvd_fail"].values

In [67]:
# COMPUTATIONAL COMPLEXITY: Reduce #samples
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(C, y_d))

In [68]:
C_ = C[tr_idx]
C_test = C[ts_idx]
#G_ = G[tr_idx]
#G_test = G[ts_idx]
#V_ = V[tr_idx]
#V_test = V[ts_idx]

In [69]:
y_d_ = y_d[tr_idx]
y_d_test = y_d[ts_idx]

y_c_ = y_c[tr_idx]
y_c_test = y_c[ts_idx]

In [70]:
ds_list = [C_]#, G_, V_]
ds_test = [C_test]#, G_test, V_test]
ds_names = ['clinic']#, 'genetic', 'vampire']

## Utilities

In [71]:
def centering_normalizing(train, test, exclusion_list = None):
    
    if exclusion_list is not None:
        scale, train = ut.centering_normalizing(train, exclusion_list)
    else:
        scale, train = ut.centering_normalizing(train)

    new_Xts = test-scale

    if exclusion_list is not None:
        new_Xts[:, exclusion_list] = test[:, exclusion_list]

    return train, new_Xts, scale


def normalizing(train, test):
    return normalize(train), normalize(test)


def make_regression(X, y, reg):

    scores = np.zeros(len(reg))
    
    for idx, r in enumerate(reg):
        model = SVC(C = r)
        scores[idx] = np.mean(cross_validate(model, X, y, return_train_score=False, cv=3)['test_score'])
        
    return reg[np.argmax(scores)]


def learn(C_, C_test, y, y_test, sparsity, centering = False, norm = False):
    if centering == True:
        C_, C_test, _ = centering_normalizing(C_, C_test)#, [5,6,7,9,10,13,15,16,17,18,19])

    if norm == True:
         C_, C_test = normalizing(C_, C_test)

    best_lambda = make_regression(C_, y, sparsity)
    model = SVC(C = best_lambda)
    model.fit(C_, y)
    y_pred = model.predict(C_test)
    #if kernel == 'linear': coef = model.coef_
    
    accuracy = ut.balanced_accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print("\tAccuracy: {}".format(accuracy))
    print("\tPrecision: {}".format(precision))
    print("\tRecall: {}".format(recall))
    #if kenel == 'linear': print("Coef: {}".format(coef))
    print("BestLambda: {}".format(best_lambda))

In [72]:
sparsity = list(np.arange(0.1, 2, 0.2))

## Basic approach

### Dementia

In [73]:
learn(C_, C_test, y_d_, y_d_test, sparsity)

	Accuracy: 0.5
	Precision: 0.0
	Recall: 0.0
BestLambda: 0.1


C:\Users\loren\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### Cardio

In [74]:
learn(C_, C_test, y_c_, y_c_test, sparsity)

	Accuracy: 0.5238813193358648
	Precision: 0.47540983606557374
	Recall: 0.11983471074380166
BestLambda: 1.5000000000000004


## Normalized data

### Dementia

In [75]:
learn(C_, C_test, y_d_, y_d_test, sparsity, norm = True)

	Accuracy: 0.5
	Precision: 0.0
	Recall: 0.0
BestLambda: 0.1


C:\Users\loren\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### Cardio

In [76]:
learn(C_, C_test, y_c_, y_c_test, sparsity, norm = True)

	Accuracy: 0.5
	Precision: 0.0
	Recall: 0.0
BestLambda: 0.1


C:\Users\loren\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Origin Data Centering

### Dementia

In [77]:
learn(C_, C_test, y_d_, y_d_test, sparsity, centering = True)

	Accuracy: 0.5
	Precision: 0.0
	Recall: 0.0
BestLambda: 0.1


C:\Users\loren\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### Cardio

In [78]:
learn(C_, C_test, y_c_, y_c_test, sparsity, centering = True)

	Accuracy: 0.5
	Precision: 0.0
	Recall: 0.0
BestLambda: 0.1


C:\Users\loren\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


## Origin Data  Centering and Normalization

### Dementia

In [79]:
learn(C_, C_test, y_d_, y_d_test, sparsity, centering = True, norm = True)

	Accuracy: 0.5
	Precision: 0.0
	Recall: 0.0
BestLambda: 0.1


C:\Users\loren\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### Cardio

In [80]:
learn(C_, C_test, y_c_, y_c_test, sparsity, centering = True, norm = True)

	Accuracy: 0.5
	Precision: 0.0
	Recall: 0.0
BestLambda: 0.1


C:\Users\loren\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
